In [1]:
import weaviate

In [14]:
client = weaviate.connect_to_local(host="localhost", port=8081)
# client = weaviate.connect_to_local(

In [15]:
import weaviate.classes as wvc

# Create the collection. Weaviate's autoschema feature will infer properties when importing.
questions = client.collections.create(
    "NSS_corpus",
    vectorizer_config=wvc.config.Configure.Vectorizer.none(),
    properties=[
        wvc.properties.Property(
            name="orig_id",
            data_type=wvc.data.DataType.int,
        ),
        wvc.properties.Property(
            name="file",
            data_type=wvc.data.DataType.string,
            index_filterable=False,
            index_searchable=False,
        ),
        wvc.properties.Property(
            name="name",
            data_type=wvc.data.DataType.string,
            index_filterable=False,
            index_searchable=False,
        ),
        wvc.properties.Property(
            name="name_lemmas",
            data_type=wvc.data.DataType.string,
            index_filterable=True,
            index_searchable=True,
        ),
        wvc.properties.Property(
            name="contents",
            data_type=wvc.data.DataType.string,
            index_filterable=False,
            index_searchable=False,
        ),
        wvc.properties.Property(
            name="contents_lemmas",
            data_type=wvc.data.DataType.string,
            index_filterable=True,
            index_searchable=True,
        ),
    ]
)

In [17]:
client.collections.list_all()

{'NSS_corpus': _CollectionConfigSimple(name='NSS_corpus', description=None, generative_config=None, properties=[], references=[], reranker_config=None, vectorizer_config=None, vectorizer=<Vectorizers.NONE: 'none'>, vector_config=None)}

In [18]:
corpus_path = "/Users/jakubkucera-sch/Documents/diplomka/llm-rag-dip/nss_chunked.jsonl"

In [19]:
import json

corpus = []
with open(corpus_path, "r") as f:
    for line in f:
        corpus.append(json.loads(line))

In [21]:
import random
corpus_s_100 = random.sample(corpus, 100)

In [22]:
corpus_s_100

[{'id': 2555,
  'file': '2053_2010.txt',
  'name': 'Právo Evropské unie: vzájemná pomoc při vymáhání finanční pohledávky; rozsah',
  'contents': 'řízení nebyly k'},
 {'id': 16287,
  'file': '2024_2010.txt',
  'name': 'Daň z příjmů: výpočet příjmu spoluvlastníka',
  'contents': 'www.nssoud.cz). Není tedy'},
 {'id': 251768,
  'file': '2021_2010.txt',
  'name': 'Kompetenční spory: příslušnost k rozhodnutí o stížnosti na postup při vyřizování žádosti o',
  'contents': 'zákona č. 114/1992 Sb.]. Pokud krajský úřad disponuje informací, že projekt trasy rychlostní'},
 {'id': 154995,
  'file': '722_2005.txt',
  'name': 'Daň z příjmů: zůstatková cena majetku',
  'contents': 'Podle § 24 odst. 2 písm. b) zákona je výdajem (nákladem) podle odstavce 1 téhož ustanovení také zůstatková cena hmotného majetku. Za zůstatkovou cenu se podle § 29 odstavce 2 zákona považuje rozdíl mezi vstupní cenou hmotného majetku a celkovou výší odpisů. Podle § 30 odst. 12 zákona pokračuje nabyvatel v odpisování započaté

In [23]:
# create emebddings of 'contents' field with huggingface
from transformers import pipeline

embedder = pipeline("feature-extraction", model="sentence-transformers/all-MiniLM-L6-v2")

/Users/jakubkucera-sch/Documents/diplomka/FlashRAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


In [27]:
from tqdm import tqdm

for item in tqdm(corpus_s_100):
    if len(item["contents"]) > embedder.tokenizer.model_max_length:
        item["contents"] = item["contents"][:embedder.tokenizer.model_max_length]
    item["embedding"] = embedder(item["contents"])[0][0]

100%|██████████| 100/100 [00:09<00:00, 10.58it/s]


In [28]:
with open("nss_chunked_100_w_embedding.jsonl", "w") as f:
    for item in corpus_s_100:
        f.write(json.dumps(item) + "\n")

In [30]:
weaviate_objs = []
for item in corpus_s_100:
    weaviate_objs.append(wvc.data.DataObject(
        # class_name="NSS_corpus",
        properties={
            "orig_id": item["id"],
            "file": item["file"],
            "name": item["name"],
            "contents": item["contents"]
            # "embedding": item["embedding"],
        },
        vector=item["embedding"],
    ))

In [31]:
nss_corpus = client.collections.get("NSS_corpus")


In [32]:
nss_corpus.data.insert_many(weaviate_objs)

BatchObjectReturn(_all_responses=[UUID('660eefa0-ca67-4899-b77a-8703f46d70ac'), UUID('8b3841d6-154b-498e-a737-fafe10ac656f'), UUID('445ea84d-3cfb-43c3-b5a8-08cdad6d072f'), UUID('ec7bc70d-b7c2-46e6-9fd5-e4c9b1a89894'), UUID('f269bc33-9e32-4002-9285-aa413e459af3'), UUID('1b0e6203-33e0-4e7f-9e54-c0380f183a0c'), UUID('aa5bcfa5-b69f-4d8d-b59d-7303f3649de8'), UUID('547f75f5-9c30-415e-90f1-ac9fa760c75e'), UUID('e5fa6477-b44e-4c8b-878d-da885c320313'), UUID('8bceadaf-872a-41b1-9358-0fe88e451ad4'), UUID('196f1f3d-7bac-443c-b02c-436061df0880'), UUID('edfccca9-0124-4d76-8f08-b2330ca09739'), UUID('2f6de47e-8098-47ff-8ce5-c0f021d44220'), UUID('4a1594d5-c9a2-4e8e-a2e3-7b1b82116878'), UUID('c702d246-a356-410d-9167-b03d928b5663'), UUID('f4237715-163b-444b-9f4b-98500f3fd166'), UUID('815df12e-3b22-4a94-910f-01a04087e402'), UUID('6722d0fa-b167-492b-807b-38a0bb7714de'), UUID('864daa8f-e381-4acd-b4de-18b22d798595'), UUID('0d485ef2-10bf-4152-a156-e9430aaf2030'), UUID('6adee027-2488-4b9f-8eb1-241a3630d827'), 

In [33]:
question = "Jaký je verdikt?"

In [35]:
question_embedding = embedder(question)[0][0]

In [40]:

response = nss_corpus.query.near_vector(
    near_vector=question_embedding,
    limit=2,
    return_metadata=wvc.query.MetadataQuery(certainty=True)
)

for o in response.objects:
    print(o.properties)

{'orig_id': 95591.0, 'file': '2001_2010.txt', 'contents': 'již zahájené', 'name': 'Řízení před soudem: rozhodnutí předběžné povahy'}
{'orig_id': 203983.0, 'file': '2349_2011.txt', 'contents': 'která měla oprávnění disponovat s prostředky na účtu zemřelého, o uhrazení vzniklého přeplatku.', 'name': '4 Ads 163/2009 - 70'}


In [62]:
response = nss_corpus.query.hybrid(
    query=question,
    # near_vector=question_embedding,
    vector=question_embedding,
    limit=2,
    return_metadata=wvc.query.MetadataQuery(certainty=True),
    # query_properties=["name^2", "contents"],
    query_properties=["name^0", "contents"],
    # alpha=1,
)
for o in response.objects:
    print(o.properties)

{'orig_id': 103632.0, 'file': '1981_2010.txt', 'contents': 'zákoníku práce se přitom za závislou práci, která je vykonávána ve vztahu nadřízenosti', 'name': 'Řízení před soudem: přezkum rozhodnutí o odvolání z funkce vedoucího státního zástupce ve'}
{'orig_id': 183782.0, 'file': '2269_2011.txt', 'contents': 'větě prvé s.ř.s. Podle tohoto ustanovení soud je vázán rozhodnutím soudů o tom, že byl spáchán', 'name': '22 Ca 177/2009 - 47'}


In [64]:
import stanza

# Download Czech models if not already downloaded
# stanza.download("cs")  # Only need to run this once

nlp = stanza.Pipeline(lang='cs', processors='tokenize,lemma,pos')

def preprocess_czech_text_stanza(text):
    doc = nlp(text)
    lemmas = []
    for sentence in doc.sentences:
        print(f"Sentence: {sentence.text}")
        print(f"Tokens: {[token.text for token in sentence.tokens]}")
        for token in sentence.tokens:
            # Each token can contain multiple words (e.g., multi-word tokens),
            # but typically it's just one. We'll take the first word for simplicity.
            if len(token.words) > 1:
                print(f"Token has multiple words: {token.words}")
            word = token.words[0]

            lemmas.append(word.lemma.lower())
            # if word.upos != 'PUNCT':  # skip punctuation
            #     # Convert lemma to lowercase
            #     lemmas.append(word.lemma.lower())
            # else:
            #     print(f"Skipping punctuation: {word.text}")
    return lemmas

# Example
text_cz = "Toto je krátká ukázka textu v češtině. Mnoho tvarů slov se liší dle pádu a rodu."
lemmas = preprocess_czech_text_stanza(text_cz)
print(lemmas)


2025-03-31 13:15:03 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-03-31 13:15:03 INFO: Downloaded file to /Users/jakubkucera-sch/stanza_resources/resources.json
2025-03-31 13:15:03 WARNING: Language cs package default expects mwt, which has been added
2025-03-31 13:15:03 INFO: Loading these models for language: cs (Czech):
| Processor | Package      |
----------------------------
| tokenize  | pdt          |
| mwt       | pdt          |
| pos       | pdt_nocharlm |
| lemma     | pdt_nocharlm |

2025-03-31 13:15:03 INFO: Using device: cpu
2025-03-31 13:15:03 INFO: Loading: tokenize
2025-03-31 13:15:03 INFO: Loading: mwt
2025-03-31 13:15:03 INFO: Loading: pos
2025-03-31 13:15:04 INFO: Loading: lemma
2025-03-31 13:15:06 INFO: Done loading processors!


Sentence: Toto je krátká ukázka textu v češtině.
Tokens: ['Toto', 'je', 'krátká', 'ukázka', 'textu', 'v', 'češtině', '.']
Sentence: Mnoho tvarů slov se liší dle pádu a rodu.
Tokens: ['Mnoho', 'tvarů', 'slov', 'se', 'liší', 'dle', 'pádu', 'a', 'rodu', '.']
['tento', 'být', 'krátký', 'ukázka', 'text', 'v', 'čeština', '.', 'mnoho', 'tvar', 'slovo', 'se', 'lišit', 'dle', 'pád', 'a', 'rod', '.']


In [53]:
for item in corpus_s_100:
   preprocessed_contents = preprocess_czech_text_stanza(item["contents"])
   print(preprocessed_contents)

Sentence: řízení nebyly k
Tokens: ['řízení', 'nebyly', 'k']
['řízení', 'být', 'k']
Sentence: www.nssoud.cz).
Tokens: ['www.nssoud.cz).']
Sentence: Není tedy
Tokens: ['Není', 'tedy']
['www.nssoud.cz).', 'být', 'tedy']
Sentence: zákona č. 114/1992 Sb.].
Tokens: ['zákona', 'č', '.', '114', '/', '1992', 'Sb', '.', ']', '.']
Sentence: Pokud krajský úřad disponuje informací, že projekt trasy rychlostní
Tokens: ['Pokud', 'krajský', 'úřad', 'disponuje', 'informací', ',', 'že', 'projekt', 'trasy', 'rychlostní']
['zákon', 'číslo', '114', '1992', 'sbírka', 'pokud', 'krajský', 'úřad', 'disponovat', 'informace', 'že', 'projekt', 'trasa', 'rychlostní']
Sentence: Podle § 24 odst. 2 písm. b) zákona je výdajem (nákladem) podle odstavce 1 téhož ustanovení také zůstatková cena hmotného majetku.
Tokens: ['Podle', '§', '24', 'odst', '.', '2', 'písm', '.', 'b', ')', 'zákona', 'je', 'výdajem', '(', 'nákladem', ')', 'podle', 'odstavce', '1', 'téhož', 'ustanovení', 'také', 'zůstatková', 'cena', 'hmotného', 'ma